<a id='Q0'></a>
<center><a target="_blank" href="http://www.propulsion.academy"><img src="https://drive.google.com/uc?id=1McNxpNrSwfqu1w-QtlOmPSmfULvkkMQV" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>

<center> <h1> Exercise:  Visualizing high-dimensional datasets</h1> </center>
<center> <h1> using Dimensionality Reduction techniques </h1> </center>

<p style="margin-bottom:1cm;"></p>

<center>Propulsion Academy, 2020</center>

Exercise II. Gene expression RNAseq
============================
Use genomic [gene expression dataset](https://drive.google.com/drive/folders/1cFaJ0Oh-8-wv7xmUoC9xvdJ5NyujTfC8?usp=sharing) (6063 samples * 14460 identifiers). This dataset is already prepared for you by removing non-active genes which reduced its size from 60499 to 14460 features. If you are a domain expert, you can clean the dataset further. Otherwise, you can use the one provided.  
The original dataset was taken from [here](https://xenabrowser.net/datapages/?dataset=tcga_RSEM_gene_tpm&host=https%3A%2F%2Ftoil.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443).  

- Visualize your dataset in a scatter plot by projecting it via t-SNE and/or UMAP, PCA or another dimensionality reduction technique into a 2-dimensional space.
- This article on [Visualizing high-dimensional datasets using PCA and t-SNE in Python](https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b) is a good starting point. Read carefully the article on [How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/), so that you don't get tricked by your plots!
 
- __Perform a principal component analysis (PCA).__
    - How many principal components represent 95% of the total variance?
    - Apply clustering after PCA. 
    - Analyze and interpret the results by creating visuals. 
    - Color the individual points from the 2-D scatter plot according to the cluster they belong to.


- Redo the clustering by using [UMAP (Uniform Manifold Approximation and Projection)](https://umap-learn.readthedocs.io/en/latest/index.html) for dimensionality reduction.

- Compare the cluster you found with the [primary_disease](https://drive.google.com/file/d/1-JrZ42o6kjK_jthPXYMdR-FY7CbqP-1m/view?usp=sharing) label from clinicalMatrix dataset. Hint: use beautiful [Sankey Diagrams](https://plotly.com/python/sankey-diagram/)
- Using the plots (scatterplot and Sankey), compare the effect of different dimensionality reduction methods on the clustering.


--------

# Notebook Set-Up

<span style="color:green">**Modules and Libraries**

In [2]:
# ----------------------------------GENERAL---------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# ----------------------------------OTHER-----------------------------------
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# ----------------------------------PIPELINE AND MODELS---------------------
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# ----------------------------------VISUALIZATION---------------------------
from sklearn.manifold import TSNE  
from umap import UMAP
from plotly import graph_objs as go
plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (12, 8)

<span style="color:green">**Necessary Functions for this Notebook**

In [3]:
def rows_and_columns_counter (df):
    print('Imported DataFrame has the following shape:', df.shape)
    print('\tso **', len(df.index),'** rows')
    print('\tso **', len(df.columns),'** columns')    

In [4]:
# FUNCTION --> Ploting distributions of sample columns
def plot_distrib(df, ncols, nrows, figsize):
    vars_ = df.columns
    c = 0
    plt.figure(figsize=figsize)
    for v in vars_:
        ax = plt.subplot(nrows, ncols, c + 1)
        _, nbins, _ = plt.hist(df[v], 20, color="g", alpha=0.6, label=v)
        plt.xlabel(v)
        plt.legend(loc="best")
        c = c + 1
    plt.show()

# Loading Data

In [6]:
gene_express = pd.read_csv('../../data/tcga_RSEM_gene_tpm_sample_dataset.csv',index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/tcga_RSEM_gene_tpm_sample_dataset.csv'

# Data Exploration

<span style="color:green">**Data Frame**

In [ ]:
#the data has more columns than rows. 
#this indicates that you should look into dimensionality reduction methods


rows_and_columns_counter (gene_express)

In [ ]:
gene_express.head(3)

<span style="color:green">**DataFrame Distributions Visualization:** Plot distribution of sample columns
    
- As there are 14460 columns, we cannot look at every feature individually.
- Here we take a random sample of only 15 columns and look at their distributions. 
- This is to get a better feel of what the data looks like. 


In [ ]:
data = gene_express.sample(n=15,axis='columns')

plot_distrib(data, ncols=5, nrows=3, figsize=(70, 50))

# Dimensionality Reduction for Visualisation

## t-SNE

- **Important note:** bear in mind that t-SNE is mainly used for visualization purposes. So you shall not include it within the Pipeline

<span style="color:green">**Scatter Visualization in 2-D:** Using t-distributed Stochastic Neighbor Embedding  **(t-SNE)**
    
- (t-SNE) is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets
- https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

TSNE Parameters:
- n_components = Dimension of the embedded space. We are projecting it onto 2 dimensions, so we can plot it in a scatter plot

- perplexity = number of nearest neighbors that is used in other manifold learning algorithms. Consider selecting a value between 5 and 50. As the perplexity value increases, we generally observe a tendency towards clearer shapes in the visualization

- n_iter = Maximum number of iterations for the optimization. Should be at least 250


In [ ]:
# t-SNE Definition: 
tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=1000)

In [ ]:
#we fit and transform high dimensional dataset to the tsne object defined above
# Fit_Transform:
X_tsne = tsne.fit_transform(gene_express)
X_tsne

In [ ]:
#X_tsne is the embedding of the training data in low-dimensional space (2 dimensions)

print('original data shape:', gene_express.shape) #(6063, 14460)
print('embedded data shape:', X_tsne.shape) #(6063, 2)

__t-SNE Visualization:__
- t-SNE maps the multi-dimensional data to a lower dimensional space and attempts to find patterns in the data by identifying observed clusters based on similarity of data points with multiple features.
- Now the data has been reduced to two dimensions, we can plot and visualize it. 

- This was not possible with the original 14460 columns

- However, after this process, the input features are no longer identifiable, and you cannot make any inference based only on the output of t-SNE. Hence it is mainly a data exploration and visualization technique.



__t-SNE compared to other techniques:__
- t-SNE does not scale well (PCA is much quicker) 
- t-SNE is for visualization purposes only. Clustering on t-SNE is a bad idea because it doesn't preserve the global data structure. Within cluster distances are  meaninful, but cluster distances between clusters are not guaranteed
- We will only do clustering with PCA


In [ ]:
# Plot Result of t-SNE:

fig = plt.figure(figsize =(15, 15))

# plot original roll
# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll
ax = fig.add_subplot()
ax.scatter(X_tsne[:, 0], X_tsne[:, 1], cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('t-SNE - Projected data')
plt.show()

## UMAP

<span style="color:green">**Scatter Visualization in 2-D:** Using Uniform Manifold Approximation and Projection **(UMAP)**
    
    
- The biggest advantage of UMAP over t-SNE is the more optimal balance between local and global structure and the computational efficiency

- https://umap-learn.readthedocs.io/en/latest/



UMAP Parameters:
- n_neighbors = parameter to control how UMAP balances local versus global structure in the data. It does this by constraining the size of the local neighborhood UMAP will look at when attempting to learn the manifold structure of the data. 

    This means that low values of n_neighbors will force UMAP to concentrate on very local structure (potentially to the detriment of the big picture), while large values will push UMAP to look at larger neighborhoods of each point when estimating the manifold structure of the data, losing fine detail structure for the sake of getting the broader of the data.

 
- n_components = the dimensionality of the reduced dimension space we will be embedding the data into


- min_dist = parameter controls how tightly UMAP is allowed to pack points together. It, quite literally, provides the minimum distance apart that points are allowed to be in the low dimensional representation. 


In [ ]:
# UMAP Definition:
umap_embeddings = UMAP(n_neighbors=30, n_components=2,  min_dist= 0.3)

In [ ]:
# Fit_Transform:
X_umap = umap_embeddings.fit_transform(gene_express)

In [ ]:
# Plot Result of UMAP:


fig = plt.figure(figsize =(15, 15))


# plot oroginal roll
# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll
ax = fig.add_subplot()
ax.scatter(X_umap[:, 0], X_umap[:, 1],cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('UMAP - Projected data')
plt.show()


#Think about how different the t-SNE and UMAP plots look, given that they are they same data!

# Dimensionality Reduction for Clustering

## Principal Component Analysis **(PCA)** 

<span style="color:green"> Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.
    
- The priority of PCA is to preserve large variances in the data using singular value decomposition. 
- It tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed


- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
# PCA Definition:
pca = PCA()

In [ ]:
# Pipeline Definition: 1º) Scaling --> 2º) Model
pipe_pca = Pipeline([("scl", StandardScaler()), ("pca", pca)])

In [ ]:
# Obtaining PCA Parameters:
pca.get_params()

In [ ]:
# Fit_transform:
gene_pca = pipe_pca.fit_transform(gene_express)

In [ ]:
#The explained_variance_ratio_ attribute in sklearn returns the percentage of variance explained by each of the selected components.
#remember we are looking for the max variance between 

pipe_pca.named_steps["pca"].explained_variance_ratio_



In [ ]:
#numpy cumsum returns the cumulative sum of the elements 
#in this case, i

evr = np.cumsum(pipe_pca.named_steps["pca"].explained_variance_ratio_)
evr

<span style="color:green">**PCA Visualization:** interceptor location

In [ ]:
# Finding out the interceptor that represents 95% of the total variance:
x_intercept = np.argmax(evr > 0.95) #evr is an array of the cumulative variances
print('The interceptor is located at component: ', x_intercept)

In [ ]:
# Ploting the location of the interceptor:

plt.plot(evr, "-x", color="lightblue") #plotting the cumulative sum of variance, so we can where the variance increases are not as pronounced
plt.xlabel("Number of Components")
plt.ylabel("Explained Variance Ratio")
plt.axvline(x=x_intercept, color="red") #vertical line to see the number of components that corresponds to the 95% total variance
plt.axhline(y=0.95, color="red") #horizontal line for 95% of the total variance 
None

<span style="color:green">**PCA Scatter Visualization in 2-D:** PCA example components
    


In [ ]:
#After dimensionality reduction, we can use gene_pca plot the principal components 

plt.rcParams["figure.figsize"] = (15, 15)

fig = plt.figure()

# # plot original roll
# ax = fig.add_subplot(311, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll (components 1 and 2)
ax = fig.add_subplot(311)
ax.scatter(gene_pca[:, 0], gene_pca[:, 1],  cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('Projected data (components 1 and 2)')

# plot projected roll (components 2 and 3)
ax = fig.add_subplot(312)
ax.scatter(gene_pca[:, 1], gene_pca[:, 2],cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('Projected data (components 2 and 3)')
plt.show()


# # plot projected roll (components 1 and 3)
# ax = fig.add_subplot(313)
# ax.scatter(gene_pca[:, 2], gene_pca[:, 3],cmap=plt.cm.Spectral)
# plt.axis('tight')
# plt.xticks([]), plt.yticks([])
# plt.title('Projected data (components 1 and 3)')
# plt.show()

plt.show()

# Clustering

<span style="color:green">**K-Means after PCA:** select the ‘optimal’ number of clusters with the **Elbow Method (for a 95% of variance)**
    
- After using PCA for dimensionality reduction, we can use k-means clustering to find the number of clusters. 
- Determining the ideal number of clusters for our k-means model can be done by measuring the sum of the squared distances to the nearest cluster center
- The k-means plot below indicates the percentage of variance explained, but in slightly different terms, as a function of the number of clusters.


In [ ]:
r_seed = 23  # random seed to use during modeling for reproducibility 
cluster_errors = []

for i in range(1, 14):
    n_clusters = i
    pipe_pca_kmean = Pipeline(
        [
            ("scl", StandardScaler()), 
            ("pca", PCA(0.95)), 
            ("cluster", KMeans(n_clusters=n_clusters, random_state=r_seed, verbose=0, n_jobs=1))]
    )

    pipe_pca_kmean.fit(gene_express)
    pipe_pca_kmean.predict(gene_express)
    cluster_errors.append(pipe_pca_kmean.named_steps["cluster"].inertia_) 

In [ ]:
plt.clf()
plt.plot(cluster_errors, "o-")
plt.xlabel("n_clusters")
plt.ylabel("sum sq distances from mean")
plt.show()

<span style="color:green">**K-Means clustering with PCA:**

In [ ]:
# Pipeline Definition: 1º) Scaling --> 2º) Model --> 3º) Clusterization Method

pipe_pca_kmean = Pipeline(
        [
            ("scl", StandardScaler()), 
            ("pca", PCA(0.95)), 
            ("cluster", KMeans(n_clusters=6, random_state=r_seed, verbose=0, n_jobs=1))]
    )


In [ ]:
# Fit:
pipe_pca_kmean.fit(gene_express)

In [ ]:
gene_kmean_cluster = pd.DataFrame(
    data = pipe_pca_kmean.named_steps["cluster"].labels_,
    index = gene_express.index,
    columns=['kmean_cluster']
)

In [ ]:
gene_kmean_cluster['kmean_cluster'].value_counts()

<span style="color:green">**Visualize cluster by t-SNE:**

In [ ]:
# tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=611)

In [ ]:
# X_tsne = tsne.fit_transform(gene_express)

In [ ]:
fig = plt.figure()

# # plot original roll
# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll
ax = fig.add_subplot()
ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c= gene_kmean_cluster['kmean_cluster'],alpha=.5, cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('t-SNE Projected data with Kmean Cluster')
plt.show()


# plt.show()

<span style="color:green">**Uniform Manifold Approximation and Projection (UMAP):** using elbow method

In [ ]:
rows_and_columns_counter (gene_express)

In [ ]:
cluster_errors = []
for i in range(1, 14):
    n_clusters = i
    pipe_umap_kmean = Pipeline(
        [
            ("scl", StandardScaler()), 
            ("UMAP", UMAP(n_neighbors=30, n_components=100,  min_dist= 0.3)), 
            ("cluster", KMeans(n_clusters=n_clusters, random_state=r_seed, verbose=0, n_jobs=1))]
    )
    pipe_umap_kmean.fit(gene_express)
    pipe_umap_kmean.predict(gene_express)
    cluster_errors.append(pipe_umap_kmean.named_steps["cluster"].inertia_) 

In [ ]:
plt.clf()
plt.plot(cluster_errors, "o-")
plt.xlabel("n_clusters")
plt.ylabel("sum sq distances from mean")
plt.show()

<span style="color:green">**K-mean after UMAP:**

In [ ]:
pipe_umap_kmean = Pipeline(
        [
            ("scl", StandardScaler()), 
            ("UMAP", UMAP(n_neighbors=30, n_components=100,  min_dist= 0.3)), 
            ("cluster", KMeans(n_clusters=6, random_state=r_seed, verbose=0, n_jobs=1))]
    )

In [ ]:
pipe_umap_kmean.fit(gene_express)

In [ ]:
gene_umap_cluster = pd.DataFrame(
    data = pipe_umap_kmean.named_steps["cluster"].labels_,
    index = gene_express.index,
    columns=['Cluster_id'])

In [ ]:
gene_umap_cluster['Cluster_id'].value_counts()

### Visualize cluster by t-SNE

In [ ]:
# tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=611)

In [ ]:
# X_tsne = tsne.fit_transform(gene_express)

In [ ]:
fig = plt.figure()

# # plot original roll
# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll
ax = fig.add_subplot()
ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c= gene_umap_cluster['Cluster_id'],alpha =0.5, cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('t-SNE Projected data with UMAP cluster')
plt.show()


# plt.show()

### Compare with given labels

In [ ]:
#these are the clusters that UMAP has given us
gene_umap_cluster.head()

In [ ]:
#phenotype_labels.csv has the true labels
phenotype = pd.read_csv('../data/phenotype_labels.csv',index_col=0)

In [ ]:
phenotype.head()

In [ ]:
le = LabelEncoder()

In [ ]:

phenotype['primary_disease_id'] = le.fit_transform(phenotype['_primary_disease'])

In [ ]:
phenotype.head()

In [ ]:
phenotype['primary_disease_id'].value_counts()

In [ ]:
le.classes_

In [ ]:
gene_umap_cluster['disease_id'] = phenotype['primary_disease_id']

In [ ]:
gene_express.head()

In [ ]:
phenotype['primary_disease_id'].reindex(gene_express.index)

In [ ]:
fig = plt.figure()

# # plot original roll
# ax = fig.add_subplot(211, projection='3d')
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
# ax.set_title("Original data")

# plot projected roll
ax = fig.add_subplot()
ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c= phenotype['primary_disease_id'].reindex(gene_express.index),alpha =0.5, cmap=plt.cm.Spectral)
plt.axis('tight')
plt.xticks([]), plt.yticks([])
plt.title('t-SNE Projected data with real data')
plt.show()


# plt.show()

In [ ]:
gene_kmean_cluster['disease_id'] = phenotype['primary_disease_id']

In [ ]:
gene_kmean_cluster.head()

### Sankey Diagram Kmean

In [ ]:
sd = gene_kmean_cluster.groupby(["kmean_cluster", "disease_id"]).size().reset_index(name="value")

In [ ]:
len(sd['disease_id'].unique())

In [ ]:
sd['kmean_cluster'] = sd['kmean_cluster']+len(sd['disease_id'].unique())

In [ ]:
sd.head()

In [ ]:
sd_labels = list(le.classes_) +['KMEAN'+str(i) for i in range(len(sd['kmean_cluster'].unique()))]

In [ ]:
sd_labels

In [ ]:
fig = go.Figure(
    data = [
    go.Sankey(
    node=dict(
        label= sd_labels
    ),
    link=dict(
        source=list(sd["disease_id"]), target=list(sd["kmean_cluster"]), value=list(sd["value"])
    ),

        )
    ]
)
       

fig.update_layout(title_text=" Kmeans Cluster Analysis", font_size=15)
fig.show()

### Sankey Diagram UMAP
<span style="color:green"> A Sankey diagram is a flow diagram, in which the width of arrows is proportional to the flow quantity.



https://plotly.com/python/sankey-diagram/

In [ ]:
sd_umap = gene_umap_cluster.groupby(["Cluster_id", "disease_id"]).size().reset_index(name="value")

In [ ]:
len(sd_umap['disease_id'].unique())

In [ ]:
sd_umap['Cluster_id'] = sd_umap['Cluster_id']+len(sd_umap['disease_id'].unique())

In [ ]:
sd_umap.head()

In [ ]:
sd_umap_labels = list(le.classes_) +['UMAP'+str(i) for i in range(len(sd_umap['Cluster_id'].unique()))]

In [ ]:
sd_umap_labels

In [ ]:
 fig = go.Figure(
    data = [
    go.Sankey(
    node=dict(
        label= sd_umap_labels
    ),
    link=dict(
        source=list(sd_umap["disease_id"]), target=list(sd_umap["Cluster_id"]), value=list(sd_umap["value"])
    ),

        )
    ]
)
       

fig.update_layout(title_text="UMAP Cluster Analysis", font_size=15)
fig.show()

--------------

# End of This Notebook